In [1]:
from  numba import njit
import numpy as np
# import pickle
from src.envs.utils import GraphDataset

In [2]:
@njit
def flatten_graph(graph):
    """
    Flatten a graph into matrices for adjacency, weights, start indices, and end indices.

    Parameters:
    - graph (adjacency matrix): The input graph to be flattened.

    Returns:
    - numpy.ndarray: Flattened adjacency matrix.
    - numpy.ndarray: Flattened weight matrix.
    - numpy.ndarray: Start indices for nodes in the flattened matrices.
    - numpy.ndarray: End indices for nodes in the flattened matrices.
    """
    flattened_adjacency = []
    flattened_weights = []
    num_nodes = graph.shape[0]
    
    node_start_indices = np.zeros(num_nodes,dtype=np.int64)
    node_end_indices = np.zeros(num_nodes,dtype=np.int64)
    
    for i in range(num_nodes):
        node_start_indices[i] = len(flattened_adjacency)
        for j in range(num_nodes):
            if graph[i, j] != 0:
                flattened_adjacency.append(j)
                flattened_weights.append(graph[i, j])
                
        node_end_indices[i] = len(flattened_adjacency)

    return (
        np.array(flattened_adjacency),
        np.array(flattened_weights),
        node_start_indices,
        node_end_indices
    )





In [3]:
@njit
def standard_greedy(graph):
    adj_matrix, weight_matrix, start_list, end_list=graph
    
    n=len(start_list)
    delta_local_cuts=np.zeros(n)
    spins=np.ones(n)
    
    
    curr_score=0
    for i in range(n):
        for j,weight in zip(adj_matrix[start_list[i]:end_list[i]],
                  weight_matrix[start_list[i]:end_list[i]]):
                
            delta_local_cuts[i]+=weight*(2*spins[i]-1)*(2*spins[j]-1)
            curr_score+=weight*(spins[i]+spins[j]-2*spins[i]*spins[j])

    curr_score/=2    
    # best_score=curr_score
    
    flag=True
    
    while flag:
        arg_gain=np.argsort(-delta_local_cuts)
        flag=False
        for v in arg_gain:
            if spins[v]:
                if delta_local_cuts[v]<0:
                    flag=False
                    break
                    
                curr_score+=delta_local_cuts[v]
                delta_local_cuts[v]=-delta_local_cuts[v]
                
                for u,weight in zip(adj_matrix[start_list[v]:end_list[v]],
                                     weight_matrix[start_list[v]:end_list[v]]):

                    delta_local_cuts[u]+=weight*(2*spins[u]-1)*(2-4*spins[v])

                spins[v] = 1-spins[v]
                flag=True
                break
                  
    return curr_score,spins

In [4]:
@njit
def mca(graph,spins):
    adj_matrix, weight_matrix, start_list, end_list=graph
    
    n=len(start_list)
    delta_local_cuts=np.zeros(n)
    
    
    
    curr_score=0
    for i in range(n):
        for j,weight in zip(adj_matrix[start_list[i]:end_list[i]],weight_matrix[start_list[i]:end_list[i]]):
                
            delta_local_cuts[i]+=weight*(2*spins[i]-1)*(2*spins[j]-1)
            curr_score+=weight*(spins[i]+spins[j]-2*spins[i]*spins[j])

    curr_score/=2   
    best_score=curr_score
    
    flag=True
    
    while flag:
        arg_gain=np.argsort(-delta_local_cuts)
        flag=False
        for v in arg_gain:
            
            if delta_local_cuts[v]<=0:
                flag=False
                break
                    
            curr_score+=delta_local_cuts[v]
            delta_local_cuts[v]=-delta_local_cuts[v]

            for u,weight in zip(adj_matrix[start_list[v]:end_list[v]],
                                 weight_matrix[start_list[v]:end_list[v]]):

                delta_local_cuts[u]+=weight*(2*spins[u]-1)*(2-4*spins[v])

            spins[v] =1-spins[v]
            flag=True
            break
                  
    return curr_score,spins

In [5]:
@njit
def tabu(graph,spins,tabu_tenure,max_steps):
    adj_matrix, weight_matrix, start_list, end_list=graph
    
    n=len(start_list)
    delta_local_cuts=np.zeros(n)
    
    tabu_list=np.ones(n)*-10000
    curr_score=0
    for i in range(n):
        for j,weight in zip(adj_matrix[start_list[i]:end_list[i]],
                  weight_matrix[start_list[i]:end_list[i]]):
                
            delta_local_cuts[i]+=weight*(2*spins[i]-1)*(2*spins[j]-1)
            curr_score+=weight*(spins[i]+spins[j]-2*spins[i]*spins[j])
            
#     print(delta_local_cuts)

    curr_score/=2    
    best_score=curr_score

    for t in range(max_steps):
        arg_gain=np.argsort(-delta_local_cuts)
        for v in arg_gain:
            if (t-tabu_list[v]> tabu_tenure) or (best_score < curr_score + delta_local_cuts[v]):

                tabu_list[v] = t

                curr_score+=delta_local_cuts[v]
                delta_local_cuts[v]=-delta_local_cuts[v]
                
                for u,weight in zip(adj_matrix[start_list[v]:end_list[v]],
                                     weight_matrix[start_list[v]:end_list[v]]):

                    delta_local_cuts[u]+=weight*(2*spins[u]-1)*(2-4*spins[v])

                spins[v] = 1-spins[v]

                break

                
        best_score=max(curr_score,best_score)
    return best_score,None
    
    

In [19]:
# graph_save_loc=f'_graphs/testing/ER_{"15-20"}spin_p{0.15}_50graphs.pkl'
# graphs=load_graph_set(graph_save_loc)

In [32]:
# test_dataset=GraphDataset('../data/testing/wishart_100vertices_m50',ordered=True)
# test_dataset=GraphDataset('../data/validation/Uniform Random-3-SAT',ordered=True)
test_dataset=GraphDataset('../data/testing/dense_MC_100_200vertices_unweighted',ordered=True)
# test_dataset=GraphDataset('../data/validation/dense_MC_100_200vertices_unweighted',ordered=True)

# test_dataset=GraphDataset('../data/validation/SK_spin_70_100vertices_weighted',ordered=True)

# test_dataset=GraphDataset('../data/testing/SK_spin_70_100vertices_weighted',ordered=True)


In [33]:
# test_dataset=GraphDataset('data/testing/Uniform Random-3-SAT',ordered=True)
tabu_cuts=[]
num_repeat=50
for i in range(len(test_dataset)):
# for i in range(100):
#     flatten_graph(graph)
    graph=test_dataset.get()
    g=flatten_graph(graph)
    best_tabu_cut=0
    for _ in range(num_repeat):
        spins= np.random.randint(2, size=graph.shape[0])
#     print(spins)
        cut,spins=tabu(g,spins,tabu_tenure=10,max_steps=graph.shape[0]*2)
        best_tabu_cut=max(best_tabu_cut,cut)
    
    tabu_cuts.append(best_tabu_cut)
    
# print('Mean Tabu Cut:',sum(tabu_cuts)/len(tabu_cuts))
tabu_cuts=np.array(tabu_cuts)

In [34]:
test_dataset.file_paths

['../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0001.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0002.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0003.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0004.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0005.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0006.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0007.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0008.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0009.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0100vertices_graph_0010.npz',
 '../data/testing/dense_MC_100_200vertices_unweighted/dense_MC_0200ver

In [35]:
# test_dataset=GraphDataset('data/testing/Uniform Random-3-SAT',ordered=True)
mca_cuts=[]
sg_cuts=[]

num_repeat=50
for i in range(len(test_dataset)):
# for i in range(100):
#     flatten_graph(graph)
    graph=test_dataset.get()
    g=flatten_graph(graph)

    best_mca_cut=0
    for _ in range(num_repeat):
        spins= np.random.randint(2, size=graph.shape[0])
    #     print(spins)
        cut,spins=mca(g,spins)
        best_mca_cut=max(cut,best_mca_cut)

    sg_cut,sg_spins=standard_greedy(g)
    
    mca_cuts.append(best_mca_cut)
    sg_cuts.append(sg_cut)

sg_cuts=np.array(sg_cuts)
mca_cuts=np.array(mca_cuts)

In [36]:
print('Tabu:',(tabu_cuts/sg_cuts).mean())
print('Max Cut Approx:',(mca_cuts/sg_cuts).mean())


Tabu: 1.0184030998658014
Max Cut Approx: 1.0138345055632405


In [52]:
tabu_cuts

array([1438., 1429., 1435., 1431., 1429., 1432., 1431., 1431., 1424.,
       1431., 5501., 5506., 5512., 5494., 5502., 5510., 5520., 5504.,
       5513., 5509.])

In [53]:
tabu_cuts/mca_cuts

array([1.00348918, 1.00351124, 1.00139567, 1.00421053, 1.00210379,
       1.00703235, 1.00210084, 1.00350631, 1.00921332, 1.00916784,
       1.00419861, 1.00200182, 1.00767824, 1.00109329, 1.00493151,
       1.00327749, 1.00363636, 1.00456288, 1.00547146, 1.00768246])

In [54]:
tabu_cuts

array([1438., 1429., 1435., 1431., 1429., 1432., 1431., 1431., 1424.,
       1431., 5501., 5506., 5512., 5494., 5502., 5510., 5520., 5504.,
       5513., 5509.])

In [55]:
mca_cuts

array([1433., 1424., 1433., 1425., 1426., 1422., 1428., 1426., 1411.,
       1418., 5478., 5495., 5470., 5488., 5475., 5492., 5500., 5479.,
       5483., 5467.])

In [56]:
sg_cuts

array([1429., 1401., 1399., 1388., 1402., 1402., 1396., 1397., 1393.,
       1407., 5411., 5421., 5416., 5416., 5428., 5418., 5439., 5409.,
       5416., 5453.])

In [57]:
import pandas as pd
import pickle

In [58]:
# Open the pickle file for reading in binary mode
with open('../data/testing/dense_MC_100_200vertices_unweighted/optimal', 'rb') as f:
# with open('../data/testing/SK_spin_70_100vertices_weighted/optimal', 'rb') as f:

    # Load the contents of the pickle file
    data = pickle.load(f)
    # data = data.sort_values(by='Instance')

In [59]:
data

,Instance,OPT
0,dense_MC_0100vertices_graph_0001,1438.0
1,dense_MC_0100vertices_graph_0002,1429.0
2,dense_MC_0100vertices_graph_0003,1435.0
3,dense_MC_0100vertices_graph_0004,1431.0
4,dense_MC_0100vertices_graph_0005,1429.0
5,dense_MC_0100vertices_graph_0006,1432.0
6,dense_MC_0100vertices_graph_0007,1431.0
7,dense_MC_0100vertices_graph_0008,1431.0
8,dense_MC_0100vertices_graph_0009,1424.0
9,dense_MC_0100vertices_graph_0010,1431.0


In [45]:
data['OPT']

0     1438.0
1     1429.0
2     1435.0
3     1431.0
4     1429.0
5     1432.0
6     1431.0
7     1431.0
8     1424.0
9     1431.0
10    5501.0
11    5506.0
12    5512.0
13    5494.0
14    5502.0
15    5510.0
16    5520.0
17    5504.0
18    5513.0
19    5509.0
Name: OPT, dtype: float64

In [81]:
s2v_cuts=[1408.0, 1405.0, 1411.0, 1420.0, 1411.0, 1404.0, 1405.0, 1404.0, 1415.0, 1391.0, 5459.0, 5435.0, 5429.0, 5454.0, 5398.0, 5425.0, 5445.0, 5439.0, 5433.0, 5463.0]
ECO_DQN_cuts=[382.0, 359.0, 377.0, 405.0, 437.0, 330.0, 373.0, 356.0, 406.0, 374.0, 267.0, 190.0, 200.0, 225.0, 203.0, 208.0, 214.0, 187.0, 206.0, 193.0, 290.0, 266.0, 272.0, 258.0, 213.0, 272.0, 269.0, 234.0, 242.0, 255.0, 330.0, 291.0, 293.0, 312.0, 343.0, 348.0, 317.0, 282.0, 384.0, 314.0]
print((ECO_DQN_cuts/data['OPT']).mean())

1.0


In [46]:
(sg_cuts/data['OPT']).mean()

0.9819577546343975

In [47]:
(tabu_cuts/data['OPT']).mean()

1.0

In [48]:
(mca_cuts/data['OPT']).mean()

0.9955127399332779

In [49]:
#dense
gflow_cuts=[1428.0, 1411.0, 1425.0, 1429.0, 1417.0, 1424.0, 1428.0, 1431.0, 1425.0, 1412.0, 5451.0, 5463.0, 5461.0, 5463.0, 5468.0, 5453.0, 5473.0, 5460.0, 5449.0, 5484.0]
# SK
# gflow_cuts=[330.0, 334.0, 334.0, 368.0, 410.0, 282.0, 348.0, 314.0, 374.0, 332.0, 257.0, 149.0, 187.0, 211.0, 175.0, 197.0, 185.0, 159.0, 199.0, 171.0, 282.0, 256.0, 250.0, 244.0, 168.0, 254.0, 214.0, 200.0, 216.0, 236.0, 315.0, 267.0, 267.0, 277.0, 305.0, 337.0, 293.0, 247.0, 367.0, 291.0]
(gflow_cuts/data['OPT']).mean()

0.9931226856207476

In [50]:
gflow_cuts/data['OPT']

0     0.993046
1     0.987404
2     0.993031
3     0.998602
4     0.991603
5     0.994413
6     0.997904
7     1.000000
8     1.000702
9     0.986723
10    0.990911
11    0.992190
12    0.990747
13    0.994357
14    0.993820
15    0.989655
16    0.991486
17    0.992006
18    0.988391
19    0.995462
Name: OPT, dtype: float64

In [51]:
data

,Instance,OPT
0,dense_MC_0100vertices_graph_0001,1438.0
1,dense_MC_0100vertices_graph_0002,1429.0
2,dense_MC_0100vertices_graph_0003,1435.0
3,dense_MC_0100vertices_graph_0004,1431.0
4,dense_MC_0100vertices_graph_0005,1429.0
5,dense_MC_0100vertices_graph_0006,1432.0
6,dense_MC_0100vertices_graph_0007,1431.0
7,dense_MC_0100vertices_graph_0008,1431.0
8,dense_MC_0100vertices_graph_0009,1424.0
9,dense_MC_0100vertices_graph_0010,1431.0


In [36]:
import pandas as pd

#SK
run_csp_cuts = [301.0, 322.0, 365.0, 308.0, 419.0, 309.0, 339.0, 320.0, 388.0, 322.0, 252.0, 179.0, 187.0, 209.0,
                194.0, 199.0, 206.0, 170.0, 179.0, 184.0, 279.0, 256.0, 254.0, 246.0, 198.0, 255.0, 236.0, 216.0,
                218.0, 233.0, 304.0, 286.0, 264.0, 279.0, 331.0, 333.0, 294.0, 257.0, 356.0, 281.0]





In [43]:
data['OPT']

20    382.0
21    359.0
22    377.0
23    405.0
24    437.0
25    330.0
26    373.0
27    356.0
28    406.0
29    374.0
0     267.0
1     190.0
2     200.0
3     225.0
4     203.0
5     208.0
6     214.0
7     187.0
8     206.0
9     193.0
30    290.0
31    266.0
32    272.0
33    258.0
34    213.0
35    272.0
36    269.0
37    234.0
38    242.0
39    255.0
10    330.0
11    291.0
12    293.0
13    312.0
14    343.0
15    348.0
16    317.0
17    282.0
18    384.0
19    314.0
Name: OPT, dtype: float64

In [41]:
(run_csp_cuts/data['OPT']).mean()

0.9216273212521102

In [42]:
run_csp_cuts/data['OPT']

20    0.787958
21    0.896936
22    0.968170
23    0.760494
24    0.958810
25    0.936364
26    0.908847
27    0.898876
28    0.955665
29    0.860963
0     0.943820
1     0.942105
2     0.935000
3     0.928889
4     0.955665
5     0.956731
6     0.962617
7     0.909091
8     0.868932
9     0.953368
30    0.962069
31    0.962406
32    0.933824
33    0.953488
34    0.929577
35    0.937500
36    0.877323
37    0.923077
38    0.900826
39    0.913725
10    0.921212
11    0.982818
12    0.901024
13    0.894231
14    0.965015
15    0.956897
16    0.927445
17    0.911348
18    0.927083
19    0.894904
Name: OPT, dtype: float64